# Topic Modelling with Latend Dirichlet Allocation

### Step 1. Load the data

In [5]:
# Loading the dataset from a csv
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

# Total number of documents
print(f'The total number of documents is: {len(documents)}')

C:\Users\User\AppData\Local\Temp\ipykernel_13400\3023278298.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);


The total number of documents is: 300000


In [6]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [7]:

# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

ModuleNotFoundError: No module named 'gensim'

In [8]:
!pip install gensim

     ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
     ---------------------------------------- 0.2/24.0 MB 4.1 MB/s eta 0:00:06
      --------------------------------------- 0.5/24.0 MB 4.7 MB/s eta 0:00:06
     - -------------------------------------- 0.7/24.0 MB 4.9 MB/s eta 0:00:05
     - -------------------------------------- 0.9/24.0 MB 4.8 MB/s eta 0:00:05
     - -------------------------------------- 1.2/24.0 MB 5.0 MB/s eta 0:00:05
     -- ------------------------------------- 1.5/24.0 MB 5.2 MB/s eta 0:00:05
     -- ------------------------------------- 1.7/24.0 MB 5.3 MB/s eta 0:00:05
     --- ------------------------------------ 2.0/24.0 MB 5.4 MB/s eta 0:00:05
     --- ------------------------------------ 2.3/24.0 MB 5.4 MB/s eta 0:00:05
     ---- ----------------------------------- 2.6/24.0 MB 5.4 MB/s eta 0:00:04
     ---- ----------------------------------- 2.8/24.0 MB 5.5 MB/s eta 0:00:04
     ----- ---------------------------------- 3.1/24.0 MB 5